In [ ]:
import pandas as pd
pd.set_option('display.max_rows', None)
import numpy  as np
from tensorflow import keras
import matplotlib.pyplot as plt

In [ ]:
# Read in the csv data using pandas
train  = pd.read_csv('../input/santander-customer-satisfaction/train.csv',index_col=0)
test   = pd.read_csv('../input/santander-customer-satisfaction/test.csv', index_col=0)
sample = pd.read_csv('../input/santander-customer-satisfaction/sample_submission.csv')

In [ ]:
train.dtypes.value_counts()

In [ ]:
train.select_dtypes(include=['int64']).nunique()

In [ ]:
features_to_drop = train.nunique()
features_to_drop = features_to_drop.loc[features_to_drop.values==1].index
# now drop these columns from both the training and the test datasets
train = train.drop(features_to_drop,axis=1)
test  = test.drop(features_to_drop,axis=1)
train.isnull().values.any()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_resampled, y_resampled,
                                                  train_size=0.5,
                                                  test_size=0.2,
                                                  random_state=42,
                                                  shuffle=True)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler  = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_val   = scaler.transform(X_val)
test    = scaler.transform(test)

In [ ]:
model = keras.Sequential(
    [
        keras.layers.Dense(units=9, activation="relu", input_shape=(X_train.shape[-1],) ),
        # randomly delete 30% of the input units below
        keras.layers.Dropout(0.3),
        keras.layers.Dense(units=9, activation="relu"),
        # the output layer, with a single neuron
        keras.layers.Dense(units=1, activation="sigmoid"),
    ]
)

# save the initial weights for later
initial_weights = model.get_weights()
model.summary()
keras.utils.plot_model(model, show_shapes=True)

In [ ]:
learning_rate = 0.001

model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
              loss="binary_crossentropy",
              metrics=keras.metrics.AUC()
             )

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(
    min_delta = 0.0002, # minimium amount of change to count as an improvement
    patience  = 20,     # how many epochs to wait before stopping
    restore_best_weights=True,
)
model.set_weights(initial_weights)

history = model.fit(X_train, y_train,
          epochs=500,
          batch_size=1000,
          validation_data=(X_val, y_val),
          verbose=0,
          # add in our early stopping callback
          callbacks=[early_stopping]
        )

In [ ]:
sample['TARGET'] = model.predict(test)